In [1]:
#import dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240


[WDM] - Driver [C:\Users\Sarah Jr\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


In [3]:
#Set URL for NASA Mars News
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [4]:
#Create a BeautifulSoup object
html = browser.html
soup = bs(html, 'html.parser')

#Extract titles
titles = soup.find_all(name='div', class_='content_title')
#Create list comprehension 
title_list = [t.text for t in titles]
#Set index for article title and store into variable
news_title = title_list[1]

#Extract paragraphs
paragraphs = soup.find_all(name='div', class_='article_teaser_body')
#Create list comprehension 
paragraph_list = [p.text for p in paragraphs]
#Set index for article teaser paragraph and store into variable
news_p = paragraph_list[0]

print(news_title)
print(news_p)

MOXIE Could Help Future Rockets Launch Off Mars
NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel.


In [5]:
#Set URL for JPL Mars Space Images - Featured Image
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)

In [6]:
#Click through full image and more info buttons to get to fullsize image
browser.links.find_by_partial_text('FULL IMAGE').click()
browser.links.find_by_partial_text('more info').click()

In [7]:
#Create a BeautifulSoup object
jpl_html = browser.html
soup = bs(jpl_html, 'html.parser')

#Find image URL
image_url = soup.find_all(name='figure', class_='lede')
image_url = image_url[0].a['href']

#Create full URL
featured_image_url = 'https://www.jpl.nasa.gov' + image_url
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17470_hires.jpg'

In [8]:
#Use Pandas to scrape Mars Facts

#set URL
url = 'https://space-facts.com/mars/'

#Read facts 
mars_facts = pd.read_html(url)[0]

#Assign column names
mars_facts.columns = ['Description','Mars']

#Set index
mars_facts.set_index('Description', inplace=True)
mars_facts

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [9]:
#Convert facts DF to html
facts = mars_facts.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n

In [10]:
#Mars Hemispheres
#Set URL
hem_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hem_url)

In [11]:
#Create empty list to store dictionaries with hemisphere titles and images
hemisphere_image_urls = []

#set variable for links to loop through 
links = browser.find_by_css("a.product-item h3")

#set loop to click each link and 
for item in range(len(links)):
    
    #click link
    browser.find_by_css('a.product-item h3')[item].click()
    
    #Find hemisphere title 
    title = browser.find_by_css('h2.title').text
    
    #Find image URL
    image = browser.find_by_css('img.wide-image')['src']
    
    
    #append title and image URL into dictionaries and main list
    hemisphere_image_urls.append({'title': title, 'image' : image})
    
    # Navigate Backwards
    browser.back()

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'image': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'image': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'image': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'image': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [12]:
browser.quit()